## XỬ LÍ - LÀM SẠCH DỮ LIỆU


#### I. Mô tả dữ liệu 

In [459]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split


In [460]:
# Đọc dữ liệu từ file csv
stroke_prediction_df = pd.read_csv('File dữ liệu thô.csv')
stroke_prediction_df.head(10)

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
5,56669,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1
6,53882,Male,74.0,1,1,Yes,Private,Rural,70.09,27.4,never smoked,1
7,10434,Female,69.0,0,0,No,Private,Urban,94.39,22.8,never smoked,1
8,27419,Female,59.0,0,0,Yes,Private,Rural,76.15,NaN,Unknown,1
9,60491,Female,78.0,0,0,Yes,Private,Urban,58.57,24.2,Unknown,1


In [461]:
# Kiểm tra thông tin sơ bộ của Data Frame
stroke_prediction_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     5110 non-null   object 
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 479.2+ KB


Dữ liệu được lấy từ bộ dữ liệu chăm sóc sức khỏe (Healthcare Dataset - Stroke Data), gồm nhiều thông tin liên quan đến hồ sơ y tế của bệnh nhân.
Một số cột quan trọng trong tập dữ liệu gồm:

- id: Mã định danh bệnh nhân (không dùng trong dự đoán).
- gender: Giới tính.
- age: Tuổi.
- hypertension: Tình trạng huyết áp cao.
- heart_disease: Bệnh tim.
- ever_married: Tình trạng hôn nhân.
- work_type: Loại công việc.
- Residence_type: Khu vực sống (nông thôn/thành thị).
- avg_glucose_level: Mức đường huyết trung bình.
- bmi: Chỉ số khối cơ thể.
- smoking_status: Tình trạng hút thuốc.
stroke: Cột mục tiêu (1 nếu có đột quỵ, 0 nếu không).

#### II. Thực hiện tiền xử lí

1. Xử lý Dữ liệu thiếu - Trùng lặp

In [462]:

# Xử lý giá trị thiếu ở cột BMI
stroke_prediction_df['bmi'].fillna(stroke_prediction_df['bmi'].median(), inplace=True)

C:\Users\hary0\AppData\Local\Temp\ipykernel_13528\2368295307.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  stroke_prediction_df['bmi'].fillna(stroke_prediction_df['bmi'].median(), inplace=True)


In [463]:
stroke_prediction_df.drop_duplicates(inplace= True)
stroke_prediction_df.duplicated().sum()

np.int64(0)

- Vì BMI là dữ liệu số, thường phân bố lệch nhẹ, nên median (trung vị) sẽ an toàn hơn mean (trung bình) khi có outlier.

2. Mã hóa dữ liệu (Encoding Categorical Data):

In [464]:
# Các cột cần mã hóa
categorical_cols = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']

# Tạo DataFrame chứa các cột đã mã hóa
encoded = pd.get_dummies(stroke_prediction_df[categorical_cols], drop_first=True)

# Ghép lại vào stroke_prediction_df gốc, giữ nguyên thứ tự các cột ban đầu
stroke_prediction_df = pd.concat([stroke_prediction_df.drop(columns=categorical_cols), encoded], axis=1)

stroke_prediction_df.head(10)


,id,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke,gender_Male,gender_Other,ever_married_Yes,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Urban,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,9046,67.0,0,1,228.69,36.6,1,True,False,True,False,True,False,False,True,True,False,False
1,51676,61.0,0,0,202.21,28.1,1,False,False,True,False,False,True,False,False,False,True,False
2,31112,80.0,0,1,105.92,32.5,1,True,False,True,False,True,False,False,False,False,True,False
3,60182,49.0,0,0,171.23,34.4,1,False,False,True,False,True,False,False,True,False,False,True
4,1665,79.0,1,0,174.12,24.0,1,False,False,True,False,False,True,False,False,False,True,False
5,56669,81.0,0,0,186.21,29.0,1,True,False,True,False,True,False,False,True,True,False,False
6,53882,74.0,1,1,70.09,27.4,1,True,False,True,False,True,False,False,False,False,True,False
7,10434,69.0,0,0,94.39,22.8,1,False,False,False,False,True,False,False,True,False,True,False
8,27419,59.0,0,0,76.15,28.1,1,False,False,True,False,True,False,False,False,False,False,False
9,60491,78.0,0,0,58.57,24.2,1,False,False,True,False,True,False,False,True,False,False,False


- Áp dụng phương pháp One-Hot Encoding để chuyển các giá trị phân loại thành các biến nhị phân (0 và 1). Mỗi giá trị trong một cột được mã hóa thành một cột mới đại diện riêng biệt.
Việc này giúp mô hình hiểu đúng bản chất của dữ liệu, tránh việc hiểu sai thứ tự giữa các giá trị (như khi dùng Label Encoding), đồng thời cải thiện khả năng học và dự đoán của mô hình.

3. Xử lý dữ liệu không liên quan (Handling Irrelevant Features):

In [465]:
# Xóa cột id không liên quan
stroke_prediction_df = stroke_prediction_df.drop( columns= 'id',axis =1, inplace = False)

- Trong bộ dữ liệu, cột id chỉ đóng vai trò là mã số định danh cho từng bản ghi (record). Nó không mang thông tin thực chất nào về đặc điểm của bệnh nhân hoặc kết quả bệnh lý, nên không có giá trị dự đoán trong mô hình học máy.

In [466]:
stroke_prediction_df.head(10)

,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke,gender_Male,gender_Other,ever_married_Yes,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Urban,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,67.0,0,1,228.69,36.6,1,True,False,True,False,True,False,False,True,True,False,False
1,61.0,0,0,202.21,28.1,1,False,False,True,False,False,True,False,False,False,True,False
2,80.0,0,1,105.92,32.5,1,True,False,True,False,True,False,False,False,False,True,False
3,49.0,0,0,171.23,34.4,1,False,False,True,False,True,False,False,True,False,False,True
4,79.0,1,0,174.12,24.0,1,False,False,True,False,False,True,False,False,False,True,False
5,81.0,0,0,186.21,29.0,1,True,False,True,False,True,False,False,True,True,False,False
6,74.0,1,1,70.09,27.4,1,True,False,True,False,True,False,False,False,False,True,False
7,69.0,0,0,94.39,22.8,1,False,False,False,False,True,False,False,True,False,True,False
8,59.0,0,0,76.15,28.1,1,False,False,True,False,True,False,False,False,False,False,False
9,78.0,0,0,58.57,24.2,1,False,False,True,False,True,False,False,True,False,False,False


#### III. Chia dữ liệu - Xử lí giá trị ngoại lai

1. Chia dữ liệu

In [467]:
# Tách dữ liệu
X = stroke_prediction_df.drop('stroke', axis=1)
y = stroke_prediction_df['stroke']

# Chia dữ liệu, lấy 20% dữ liệu để làm test, 80% dữ liệu để train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


2. Xử lí ngoại lai trên tập train


In [468]:
# Chọn các cột numeric
numeric_cols = ['age', 'avg_glucose_level', 'bmi']

# Tính Q1, Q3 và IQR
Q1 = X_train[numeric_cols].quantile(0.25)
Q3 = X_train[numeric_cols].quantile(0.75)
IQR = Q3 - Q1

# Giới hạn giá trị bằng clip theo cột (axis=1)
X_train[numeric_cols] = X_train[numeric_cols].clip(lower=Q1 - 1.5 * IQR, 
                                                   upper=Q3 + 1.5 * IQR, 
                                                   axis=1)


- Xử lý giá trị ngoại lai (outlier) giúp mô hình học máy hoạt động ổn định và chính xác hơn. Vì các giá trị quá lớn hoặc quá nhỏ có thể làm sai lệch trung bình, phương sai và hướng học của mô hình, dẫn đến kết quả dự đoán kém tin cậy.

#### III. Chuẩn hóa và cân bằng dữ liệu

1. Chuẩn hóa dữ liệu

In [469]:

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train[numeric_cols])

# Chuyển về DataFrame giữ index
X_train_scaled = pd.DataFrame(X_train_scaled, columns=numeric_cols, index=X_train.index)

# Kết hợp với các cột categorical còn lại
categorical_cols = [col for col in X_train.columns if col not in numeric_cols + ['stroke']]

X_train_scaled = pd.concat([X_train_scaled, X_train[categorical_cols]], axis=1)

X_train_scaled.head(10)


,age,avg_glucose_level,bmi,hypertension,heart_disease,gender_Male,gender_Other,ever_married_Yes,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Urban,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
845,0.205661,-0.960346,0.615716,0,0,False,False,True,False,True,False,False,True,False,True,False
3744,-1.254901,0.632522,-1.079822,0,0,True,False,False,False,True,False,False,False,False,True,False
4183,1.046590,0.277250,-0.533326,0,0,False,False,True,False,False,True,False,False,False,True,False
3409,0.028623,-1.074466,-0.547339,0,0,True,False,True,False,True,False,False,True,False,False,True
284,-1.299160,-0.566033,0.405525,0,0,True,False,False,False,False,False,False,True,False,False,False
4796,1.312147,-0.897881,1.260301,1,0,True,False,False,False,False,False,False,True,False,True,False
1283,0.869552,1.479409,2.213165,1,1,True,False,True,False,True,False,False,True,True,False,False
3656,-0.059896,-0.776253,1.302339,0,0,False,False,True,False,False,True,False,False,False,True,False
2485,-1.299160,-0.920404,-0.603390,0,0,False,False,False,False,True,False,False,False,True,False,False
1282,0.426958,2.052258,0.461576,0,1,True,False,True,False,True,False,False,False,False,False,False


- Việc chuẩn hóa các cột numeric như age, avg_glucose_level và bmi là bước quan trọng trong tiền xử lý dữ liệu, giúp đưa tất cả các biến về cùng thang đo với trung bình gần 0 và độ lệch chuẩn bằng 1. Điều này rất cần thiết vì các biến ban đầu có phạm vi và đơn vị khác nhau, nếu không chuẩn hóa, những biến có giá trị lớn sẽ chi phối mô hình, làm thuật toán học máy nhạy cảm với các giá trị này.

2. Cân bằng dữ liệu bằng phương pháp SMOTE (Synthetic Minority Oversampling Technique)

- Trong tập dữ liệu này, số lượng người không bị đột quỵ (stroke = 0) nhiều hơn rất nhiều so với người bị đột quỵ (stroke = 1).
Nếu không cân bằng, mô hình sẽ có xu hướng dự đoán nghiêng về lớp chiếm đa số, dẫn đến độ chính xác ảo nhưng không phát hiện được các trường hợp đột quỵ thật.

In [470]:
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_scaled, y_train)

# Kiểm tra số lượng mỗi lớp sau khi cân bằng
pd.Series(y_train_balanced).value_counts()


stroke
0    3889
1    3889
Name: count, dtype: int64

In [471]:
# Gộp dữ liệu lại thành Data Frame để thực hiện thống kê mô tả, dễ quan sát

stroke_prediction_train = pd.DataFrame(X_train_balanced, columns=X.columns)
stroke_prediction_train['stroke'] = y_train_balanced

# Xóa các dữ liệu trùng lặp sau khi thực hiện cân bằng
stroke_prediction_train.drop_duplicates(inplace=True)
stroke_prediction_train.duplicated().sum()

np.int64(0)

3. Thống kê mô tả (Dựa trên dữ liệu cân bằng của tập train)

In [472]:
stroke_prediction_train.describe()

,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke
count,7778.000000,7778.000000,7778.000000,7778.000000,7778.000000,7778.000000
mean,0.516994,0.118925,0.070969,0.238974,0.084103,0.500000
std,0.958569,0.323721,0.256790,1.161189,0.872593,0.500032
min,-1.915251,0.000000,0.000000,-1.383489,-2.579182,0.000000
25%,-0.015636,0.000000,0.000000,-0.694251,-0.421522,0.000000
50%,0.733213,0.000000,0.000000,-0.166041,-0.052288,0.500000
75%,1.312147,0.000000,0.000000,1.572625,0.517627,1.000000
max,1.710482,1.000000,1.000000,2.052258,2.465394,1.000000


- Thống kê mô tả của bộ dữ liệu cho thấy các đặc trưng đã được chuẩn hóa và cân bằng đầy đủ. Các biến liên tục như age, avg_glucose_level và bmi có giá trị trung bình gần 0 và độ lệch chuẩn xấp xỉ 1, với các giá trị cực đại và cực tiểu nằm trong khoảng ±3 SD, cho thấy phân phối dữ liệu hợp lý và các outlier đã được xử lý. Các biến nhị phân hypertension và heart_disease có phần lớn giá trị là 0, phản ánh tỉ lệ mắc bệnh thấp trong tập dữ liệu, trong khi nhãn stroke đã được cân bằng hoàn toàn, trung bình = 0.5, độ lệch chuẩn ≈ 0.5, đảm bảo sự cân bằng giữa các lớp. Nhìn chung, thống kê mô tả xác nhận dữ liệu đã sạch, chuẩn hóa tốt và sẵn sàng cho các bước phân tích hoặc xây dựng mô hình học máy.